In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import matplotlib.pyplot as plt
import pandas as pd
import random
import graphviz
import seaborn as sns
import time
import math
from torch.nn.utils import clip_grad_norm_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
path_1 = "/content/drive/MyDrive/Monarch/RL/Tic Tac Toe/Deep Q Learning/nn_training_data_first_half.csv"
path_2 = "/content/drive/MyDrive/Monarch/RL/Tic Tac Toe/Deep Q Learning/nn_training_data_second_half.csv"

offline_TTT_trajectories_first_half = pd.read_csv(path_1).to_numpy()
offline_TTT_trajectories_second_half = pd.read_csv(path_2).to_numpy()

offline_TTT_trajectories = np.concatenate((offline_TTT_trajectories_first_half, offline_TTT_trajectories_second_half), axis=0)
print(offline_TTT_trajectories.shape)

In [ ]:
# 1. Define your environment
class TicTacToe:
  def __init__(self):
    self.board = np.zeros((3, 3))

  def reset(self):
    self.board = np.zeros((3, 3))
    return self.board

  def check_win(self, player):
    # Check rows
    for i in range(3):
      if np.all(self.board[i, :] == player):
        return player

    # Check columns
    for j in range(3):
      if np.all(self.board[:, j] == player):
        return player

    #Check diagonal
    if np.all(np.diag(self.board) == player):
      return player
    if np.all(np.diag(np.fliplr(self.board)) == player):
      return player

    #Check tie
    if np.all(self.board != 0):
      return -1

    return 0

  # Check for empty places on board
  def valid_state(self, potential_action):
    valid_moves = []
    for i in range(len(self.board)):
        for j in range(len(self.board)):

            if self.board[i][j] == 0:
                valid_moves.append((i, j))
    if potential_action in valid_moves:
        return True
    else:
        return False

  # Check for empty places on board
  def possibilities(self):
    l = []
    print(self.board)
    for i in range(len(self.board)):
        for j in range(len(self.board)):

            if self.board[i][j] == 0:
                l.append((i, j))
    return(l)

  def random_step(self, player):
    selection = self.possibilities()
    current_loc = random.choice(selection)
    self.board[current_loc] = player
    done = self.check_win(player)
    reward = 0
    if done == 2:
      reward = -1
    return self.board, reward, done

  def network_step(self, action, player):
    #print(action)
    row, col = action
    self.board[row, col] = player
    done = self.check_win(player)
    reward = 0
    if done == 1:
      reward = 1
    return self.board, reward, done



In [ ]:
class TransformerAgent(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim, nhead, num_layers, dropout=0.1):
        super(TransformerAgent, self).__init__()
        self.transformer = nn.Transformer(d_model=hidden_dim, nhead=nhead,
                                          num_decoder_layers=num_layers,
                                          num_encoder_layers=num_layers,
                                          dropout=dropout)
        # Instead of an Embedding layer, use a Linear layer to handle continuous state values
        self.state_embedding = nn.Linear(state_dim, hidden_dim)
        self.actor = nn.Linear(hidden_dim, action_dim)
        self.state_dim = state_dim

    def forward(self, x):
        # Embed the input using the Linear layer
        x = self.state_embedding(x.float())
        # Reshape to (sequence_length, batch_size, embedding_dim)
        x = x.view(1, 1, -1) # Reshape for Transformer

        # Pass through the Transformer
        output = self.transformer(x, x)[0, -1, :]
        # Actor output (logits)
        logits = self.actor(output)
        return logits

In [ ]:
tensor_to_tuple = {}
for i in range(9):
  tensor_to_tuple[i] = (i//3, i%3)
print(tensor_to_tuple)

In [ ]:
from logging import log
#Tic Tac Toe version
def compute_discounted_rewards(rewards, gamma=0.99):
    discounted_rewards = []
    discounted_reward = 0
    i = 0
    for reward in reversed(rewards):
        discounted_reward = reward + gamma * discounted_reward
        discounted_rewards.insert(0, discounted_reward)
        #print(i," discounted_rewards: ", discounted_rewards)
        i += 1
    discounted_rewards = torch.tensor(discounted_rewards)

    discounted_rewards = discounted_rewards / (discounted_rewards.std() + 1e-5)
    #discounted_rewards = discounted_reward - discounted_rewards.mean() / (discounted_rewards.std() + 1e^-5)
    # print("discounted rewards: ", discounted_rewards)
    # print("discounted rewards mean: ", discounted_rewards.mean())
    # print("discounted rewards std: ", discounted_rewards.std())

    return discounted_rewards

#%%debug
debug = False

def train(env, policy, optimizer, episodes=2000001):

  i = 0
  games_won = 0
  games_lost = 0
  games_skipped = 0
  for episode in range(episodes):
    print("trajectory: ", i)
    state = env.reset()
    log_probs = []
    rewards = []


    done = 0
    not_done_counter = 0
    while done == 0:
        print(state)
        state = state.flatten()
        state = torch.FloatTensor(state).unsqueeze(0)
        if debug:
          print("inner episode loop ", not_done_counter, " state: ", state)

        probs = policy(state)
        if debug:
          print("inner episode loop ", not_done_counter, " probs: ", probs)

        action_mask = torch.zeros_like(probs)
        for i in range(len(action_mask[0])): # Iterate through the actions in the mask
          if env.valid_state(tensor_to_tuple[i]):
            action_mask[0][i] = 1 # Apply mask to the specific action
        action_probabilities = Categorical(probs * action_mask)
        if debug:
          print("inner episode loop ", not_done_counter, " action_probabilities: ", action_probabilities)

        action = action_probabilities.sample()
        if debug:
          print("inner episode loop ", not_done_counter, " action: ", action)
          print("")
          print("")
          print("")

        if debug:
          print("inner episode loop ", not_done_counter, "step input action.item(): ", action.item())

        state, reward, done = env.network_step(tensor_to_tuple[action.item()], player = 1)

        #Only generate random move is state is not terminal
        if done == 0:
          state, reward, done = env.random_step(player = 2)
        if debug:
          print("inner episode loop after step ", not_done_counter)
          print("step output state: \n", state)
          print("step output reward: ", reward)
          print("step output done: ", done)
          print("")
          print("")
          print("")



        log_probs.append(action_probabilities.log_prob(action))
        rewards.append(reward)

        # Inside the train function, after an episode ends:
        if done != 0:
            i += 1
            # print(state)
            print("DONE!!! ", done)
            if sum(rewards) == 1:
              games_won += 1
            elif sum(rewards) == -1:
              games_lost += 1
            episode_rewards.append(sum(rewards))
            discounted_rewards = compute_discounted_rewards(rewards)

            policy_loss = []
            for log_prob, Gt in zip(log_probs, discounted_rewards):
                policy_loss.append(-log_prob * Gt)
            optimizer.zero_grad()
            policy_loss = torch.cat(policy_loss).sum()
            policy_loss.backward()
            optimizer.step()
            episode_losses.append(policy_loss.item())


            if episode % 50 == 0:
                print(f"Episode {episode}, Total Reward: {sum(rewards)}")
                print("games_won: ", games_won)
                print("games lost: ", games_lost)
                print("games_skipped: ", games_skipped)
                print("")
                print("")
                print("")

            break



In [ ]:
#Globals
episode_rewards = []
episode_losses = []

#Driver code for training the model
my_env = TicTacToe()
print("")
policy = PolicyNetwork()
print("Policy: ", policy)
total_params = sum(p.numel() for p in policy.parameters())
print(f"Number of parameters: {total_params}")
optimizer = optim.Adam(policy.parameters(), lr=1e-6)

#train(env, policy, optimizer)
train(my_env, policy, optimizer)